In [12]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import os
from functools import reduce

In [58]:
csvpath = "Resources/netflix_titles.csv"
netflix_title_df= pd.read_csv(csvpath)
netflix_title_df = netflix_title_df.rename(columns={'title': 'Title'})
netflix_title_df.head()


,show_id,type,Title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [14]:
url = 'https://en.wikipedia.org/wiki/List_of_Netflix_original_programming'

In [15]:
tables = pd.read_html(url)
tables

[                         Title                               Genre  \
 0              Stranger Things              Science fiction/horror   
 1                    The Crown                    Historical drama   
 2                        Ozark                         Crime drama   
 3                Lost in Space                     Science fiction   
 4               Narcos: Mexico                         Crime drama   
 5         The Umbrella Academy                    Superhero action   
 6                 Black Summer                        Zombie drama   
 7                 Another Life               Science fiction drama   
 8                 Criminal: UK  Police procedural anthology series   
 9                 Raising Dion     Superhero/Science fiction drama   
 10                Virgin River                      Romantic drama   
 11                 The Witcher                       Fantasy drama   
 12                 Locke & Key                   Horror teen drama   
 13   

In [46]:
df_dramas = tables[0]
df_comedies = tables[1]
df_adultanim = tables[2]
df_anime = tables[3]
df_arb = tables[5]
df_danish = tables[6]
df_french = tables[7]
df_german = tables[8]
df_hindi = tables[9]
df_italian = tables[10]
df_japanese = tables[11]
df_korean= tables[12]
df_nor = tables[13]
df_polish = tables[14]
df_port = tables[15]
df_spanish = tables[16]
df_swedish = tables[17]
df_tamil = tables[18]
df_turkish = tables[19]
df_other = tables[20]
df_docu = tables[21]
df_reality = tables[22]

In [57]:
df_final = pd.merge(df_dramas, df_comedies, how='outer')
df_final = pd.merge(df_final,df_adultanim, how='outer')
df_final = pd.merge(df_final,df_anime, how='outer')
df_final = pd.merge(df_final,df_arb, how='outer')
df_final = pd.merge(df_final,df_danish, how='outer')
df_final = pd.merge(df_final,df_french, how='outer')
df_final = pd.merge(df_final,df_german, how='outer')
df_final = pd.merge(df_final,df_hindi, how='outer')
df_final = pd.merge(df_final,df_italian, how='outer')
df_final = pd.merge(df_final,df_japanese, how='outer')
df_final = pd.merge(df_final,df_korean, how='outer')
df_final = pd.merge(df_final,df_nor, how='outer')
df_final = pd.merge(df_final,df_polish, how='outer')
df_final = pd.merge(df_final,df_port, how='outer')
df_final = pd.merge(df_final,df_spanish, how='outer')
df_final = pd.merge(df_final,df_swedish, how='outer')
df_final = pd.merge(df_final,df_tamil, how='outer')
df_final = pd.merge(df_final,df_turkish, how='outer')
df_final = pd.merge(df_final,df_other, how='outer')
df_final = pd.merge(df_final,df_docu, how='outer')
df_final = pd.merge(df_final,df_reality, how='outer')
del df_final['Language']
del df_final['Unnamed: 6']
del df_final['Unnamed: 7']
df_final.head()

,Title,Genre,Premiere,Seasons,Runtime,Status
0,Stranger Things,Science fiction/horror,"July 15, 2016","3 seasons, 25 episodes",42–78 min.,Season 4 due to premiere in 2022[1]
1,The Crown,Historical drama,"November 4, 2016","4 seasons, 40 episodes",47–61 min.,Renewed for seasons 5 and 6 (final)[2][3]
2,Ozark,Crime drama,"July 21, 2017","3 seasons, 30 episodes",52–80 min.,Renewed for two part final season[4]
3,Lost in Space,Science fiction,"April 13, 2018","2 seasons, 20 episodes",39–66 min.,Renewed for final season[5]
4,Narcos: Mexico,Crime drama,"November 16, 2018","2 seasons, 20 episodes",45–70 min.,Renewed[6]


In [59]:
netflix_title_df
wiki_kaggle_df = pd.merge(netflix_title_df,df_final, how='inner', on='Title')
wiki_kaggle_df.head()

,show_id,type,Title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Seasons,Runtime,Status
0,s219,TV Show,A Queen Is Born,Carla Barros,"Gloria Groove, Alexia Twister",Brazil,"November 11, 2020",2020,TV-14,1 Season,"International TV Shows, Reality TV",Gloria Groove and Alexia Twister make drag dre...,Reality,"November 11, 2020","1 season, 6 episodes",42–44 min.,Pending
1,s325,TV Show,After Life,NaN,"Ricky Gervais, Tom Basden, Tony Way, Diane Mor...",United Kingdom,"April 24, 2020",2020,TV-MA,2 Seasons,"British TV Shows, International TV Shows, TV C...",Struggling to come to terms with his wife's de...,Comedy,"March 8, 2019","2 seasons, 12 episodes",25–31 min.,Renewed[40]
2,s342,TV Show,Aggretsuko,NaN,"Kaolip, Komegumi Koiwasaki, Maki Tsuruta, Soht...",Japan,"August 27, 2020",2020,TV-14,3 Seasons,"Anime Series, International TV Shows","Frustrated with her thankless office job, Rets...",Workplace comedy,"April 20, 2018","3 seasons, 30 episodes",15–23 min.,Renewed[58]
3,s394,TV Show,Alice in Borderland,NaN,"Kento Yamazaki, Tao Tsuchiya, Nijiro Murakami,...",Japan,"December 10, 2020",2020,TV-MA,1 Season,"International TV Shows, TV Action & Adventure,...",An aimless gamer and his two friends find them...,Fantasy,"December 10, 2020","1 season, 8 episodes",41–52 min.,Renewed[95]
4,s399,TV Show,Alien Worlds,NaN,NaN,United Kingdom,"December 2, 2020",2020,TV-PG,1 Season,"British TV Shows, Docuseries, International TV...",Applying the laws of life on Earth to the rest...,Docu-series,"December 2, 2020","1 season, 4 episodes",41–47 min.,Pending
